In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import sys

sys.path.append('..')

from benchmark.himmelblau import Himmelblau
from benchmark.holder import Holder
from benchmark.rosenbrock import Rosenbrock
from benchmark.michalewicz import Michalewicz

from optims.BayesOpt import BayesOpt
from optims.AdaLIPO_E import AdaLIPO_E
from optims.SBS import SBS

In [2]:
def create_bounds(min, max, dim):
    bounds = [(min, max) for _ in range(dim)]
    return np.array(bounds)

In [3]:
def time_it (func, *args):
    import time
    start = time.time()
    res = func(*args)
    end = time.time()
    return res, end-start

In [4]:
fs = {
  "Himmelblau": [Himmelblau(), create_bounds(-4, 4, 2)],
  "Holder Table": [Holder(), create_bounds(-10, 10, 2)],
  "Michalewicz": [Michalewicz(), create_bounds(0, np.pi, 2)],
  "Rosenbrock": [Rosenbrock(), create_bounds(-3, 3, 2)],
}

evals = np.linspace(5, 100, 10, dtype=int)

bayes_results = {}
bayes_times = {}

for f_name in tqdm(fs.keys()):
  h = fs[f_name][0]
  bounds = fs[f_name][1]

  bayes_results_tmp = []
  bayes_times_tmp = []
  for num_evals in evals:
    bayes = BayesOpt(bounds, n_iter=num_evals)
    res, time = time_it(bayes.optimize, h)
    opt = res[0][1]
    bayes_results_tmp.append(np.abs(h.min - opt))
    bayes_times_tmp.append(time)
    
  bayes_results[f_name] = bayes_results_tmp
  bayes_times[f_name] = bayes_times_tmp


In [ ]:
for f_name in fs.keys():
  plt.plot(evals, bayes_results[f_name])
  plt.yscale("log")
  plt.xlabel("Number of evaluations")
  plt.ylabel("log distance to minimum")
  plt.title(f"Convergence of BayesOpt on {f_name}")
  plt.savefig(f"fig/perf/perf_vs_evals_bayes_{f_name}.png", bbox_inches="tight")
  plt.clf()

<Figure size 640x480 with 0 Axes>

In [ ]:
for f_name in fs.keys():
  plt.plot(evals, bayes_times[f_name])
  plt.xlabel("Number of evaluations")
  plt.ylabel("time (s)")
  plt.title(f"Time taken by BayesOpt on {f_name}")
  plt.savefig(f"fig/time/time_vs_evals_bayes_{f_name}.png", bbox_inches="tight")
  plt.clf()

<Figure size 640x480 with 0 Axes>

In [ ]:
evals = np.linspace(300, 2000, 10, dtype=int)

ada_times = {}

for f_name in tqdm(fs.keys()):
  h = fs[f_name][0]
  bounds = fs[f_name][1]

  ada_times_tmp = []
  for num_evals in evals:
    ada = AdaLIPO_E(bounds, max_evals=num_evals)
    res, time = time_it(ada.optimize, h)
    ada_times_tmp.append(time)
    
  ada_times[f_name] = ada_times_tmp


  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
for f_name in fs.keys():
  plt.plot(evals, ada_times[f_name])
  plt.xlabel("Number of evaluations")
  plt.ylabel("time (s)")
  plt.title(f"Time taken by AdaLIPO on {f_name}")
  plt.savefig(f"fig/time/time_vs_evals_ada_{f_name}.png", bbox_inches="tight")
  plt.clf()

<Figure size 640x480 with 0 Axes>

In [ ]:
evals = np.linspace(300, 1_000_000, 10, dtype=int)

sbs_times = {}

for f_name in tqdm(fs.keys()):
  h = fs[f_name][0]
  bounds = fs[f_name][1]

  sbs_times_tmp = []
  for num_evals in evals:
    svgd_iter = 300
    n_particles = max(num_evals // (300 * 3), 1)
    sbs = SBS(
      bounds,
      n_particles=n_particles,
      k_iter=[10_000],
      svgd_iter=svgd_iter,
      sigma=1 / n_particles**2,
      lr=0.2
    )
    res, time = time_it(sbs.optimize, h)
    sbs_times_tmp.append(time)
    
  sbs_times[f_name] = sbs_times_tmp


  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
for f_name in fs.keys():
  plt.plot(evals, sbs_times[f_name])
  plt.xlabel("Number of evaluations")
  plt.ylabel("time (s)")
  plt.title(f"Time taken by SBS on {f_name}")
  plt.savefig(f"fig/time/time_vs_evals_sbs_{f_name}.png", bbox_inches="tight")
  plt.clf()

<Figure size 640x480 with 0 Axes>